In [186]:
import urllib.request
import json
from bs4 import BeautifulSoup
import re
import json
import pymongo


URL = "http://www.flipkart.com"

def mongodbConnection():
    conn = pymongo.MongoClient("mongodb://localhost:27017")
    mydb = conn["Reviews"]
    mycol = mydb["FlipkartReview"]
    return mycol                                         
    
def get_html(url):
    fp = urllib.request.urlopen(url)
    mybytes = fp.read()

    mystr = mybytes.decode("utf8")
    fp.close()
    return mystr


def remove_non_ascii(mystring):
    return re.sub(r'[^\x00-\x7f]',r'', mystring) 

def get_reviews(mystring):
    query = {"Product":mystring}
    mycol = mongodbConnection()
    mydoc = mycol.find(query,{'_id': False})
    if(mydoc.explain()['executionStats']['nReturned'] != 0):
        return mydoc
        for x in mydoc:
            print(x)
            print("-----------------------")
    
    else:
        myReviews = []
        PAGE = 1
        PAGENEXT = True
        url = URL +"/search?q="+mystring.replace(" ","%20")
        mystr = get_html(url)
        e = 0
        soup = BeautifulSoup(mystr)
        links = soup.findAll(name="script",id="jsonLD")
        links_dict = json.loads(links[0].string)

        soup = BeautifulSoup(get_html(links_dict['itemListElement'][0]['url']))
        links_dict['itemListElement'][0]
        base_url = links_dict['itemListElement'][0]['url'].split("/p/")[0]
        soup.find_all("a", href=re.compile("product-reviews/"))
        try:
            product_review_url = "https://www.flipkart.com"+soup.find_all("a", href=re.compile("product-reviews/"))[0].get("href").split(sep="&lid=")[0]
        except:
            return "review not found"

        while PAGENEXT:
            print("Getting Page {}".format(PAGE))
            soup = BeautifulSoup(get_html(product_review_url))
            mydict = soup.findAll(name="script",id="is_script")

            json_text = re.search(r'^\s*window\.__INITIAL\_STATE__\s*=\s*({.*?})\s*;\s*$',
                                  mydict[0].string, flags=re.DOTALL | re.MULTILINE).group(1)

            script_dictionary = json.loads(json_text)

            list_of_reviews = [value for value in script_dictionary['pageDataV4']['page']['data']['10002'] if value['slotType']=="WIDGET" if (value['widget']['type'] == "REVIEWS")]

            if(len(list_of_reviews) == 0):
                list_of_reviews = [value for value in script_dictionary['pageDataV4']['page']['data']['10002'] if value['slotType']=="WIDGET" if (value['widget']['type'] == "ASPECT_REVIEWS")]
            #return list_of_reviews 
            for reviews in list_of_reviews:
                Reviews_Dict = {"id":0,"Product":"", "Name":"","Rating":0,"Comment Heading":"","Comments":""}
                Reviews_Dict['id'] = e
                Reviews_Dict['Product'] = mystring
                Reviews_Dict['Name'] = reviews['widget']['data']['renderableComponents'][0]['value']['author']
                Reviews_Dict["Comment Heading"] = reviews['widget']['data']['renderableComponents'][0]['value']['title']
                Reviews_Dict["Rating"] = reviews['widget']['data']['renderableComponents'][0]['value']['rating']
                Reviews_Dict["Comments"] = reviews['widget']['data']['renderableComponents'][0]['value']['text']

                Reviews_Dict['Name'] = remove_non_ascii(Reviews_Dict['Name'])
                Reviews_Dict['Comment Heading'] = remove_non_ascii(Reviews_Dict['Comment Heading'])
                Reviews_Dict['Comments'] = remove_non_ascii(Reviews_Dict['Comments'])
                myReviews.append(Reviews_Dict)
                e += 1
                #print(myReviews)
            #return myReviews  
            PAGE = PAGE + 1
            nextpage = "page={}".format(PAGE)
            mydata = soup.findAll(name = "a", href=re.compile(nextpage))
            if(len(mydata) == 0):
                PAGENEXT = False

            else:    
                for data in soup.findAll(name = "a", href=re.compile(nextpage)):
                    if(data.findAll(name="span", text="Next")):
                        product_review_url = URL + data.get("href")
                        PAGENEXT = True
                        break
                    else:
                        PAGENEXT = False   
        mycol.insert_many(myReviews)
        mydoc = mycol.find(query,{'_id': False})
        return mydoc
        for x in mydoc:
            print(x)
            print("-----------------------")
        return myReviews


In [187]:
reviews = get_reviews("iphone 11")

In [188]:
reviews

In [183]:
reviews = get_reviews("samsung a8")

Getting Page 1
Getting Page 2
Getting Page 3
Getting Page 4
Getting Page 5
Getting Page 6
Getting Page 7
Getting Page 8
Getting Page 9
Getting Page 10
Getting Page 11
Getting Page 12
Getting Page 13
Getting Page 14
Getting Page 15
Getting Page 16
Getting Page 17
Getting Page 18
Getting Page 19
Getting Page 20
Getting Page 21
Getting Page 22
Getting Page 23
Getting Page 24
Getting Page 25
{'id': 0, 'Product': 'samsung a8', 'Name': 'Soumya Guha Roy', 'Rating': 4, 'Comment Heading': 'Beauty & the beast combined.', 'Comments': "I have been using the device since last 3 days. Here is my review \n\nPros: \n       1. Excellent display \n       2. Fast focus and fast capture camera\n       3. Nice UI\n       4. Decent battery backup (after 24 hours, battery remains ~30% with medium\n           usage.\n       5. Good performance.\n       6. Unread notification count is spot on, like we have in iOS.\n       7. The metallic design is superb, and feels premium. Go for the gold one, than the\n     

In [152]:
len(reviews)

10

In [112]:
reviews[2]['widget']['data']['renderableComponents'][0]['value']['author']

'deepak pagar nashik'

In [120]:
for review in reviews:

    print(review['widget']['data']['renderableComponents'][0]['value']['author'])
    print("------------------")
    
    

Soumya Guha Roy
------------------
Karthi
------------------
deepak pagar nashik
------------------
Arun Nath S
------------------
Rejo Raju
------------------
Vimal  S
------------------
Sumanth Babu
------------------
VINIT KUMAR
------------------
Archita Jalan
------------------
Anoop Kumar Gupta
------------------


In [88]:
reviews[0]['widget']['data']['renderableComponents']

[{'action': None,
  'metaData': None,
  'tracking': {'position': '1',
   'reviewLanguage': 'en',
   'impressionId': 'f350e9f4-083a-43b3-a5ca-7802fd5e9164',
   'contextType': 'page',
   'otracker': '1'},
  'trackingData': None,
  'value': {'type': 'ProductReviewValue',
   'author': 'Soumya Guha Roy',
   'certifiedBuyer': True,
   'created': 'Sep, 2015',
   'downvote': {'action': {'constraints': [{'metaInfo': {'openInBottomSheet': 'true',
        'url': '/login?type=mobile&sourceContext=review_vote'},
       'type': 'LOGIN'}],
     'fallback': None,
     'loginType': 'LEGACY_LOGIN',
     'omnitureData': None,
     'originalUrl': None,
     'params': {'reviewDomain': 'PRODUCT',
      'reviewLanguage': 'en',
      'source': 'prp',
      'reviewId': '49708745-33a2-4af3-9146-c8c50f3ce40e',
      'vote': 'DOWN'},
     'requiredPermissionType': None,
     'screenType': None,
     'tracking': {},
     'type': 'REVIEW_VOTE',
     'url': None,
     'validationMeta': None},
    'metaData': None,
 

In [81]:
len(reviews)

241

In [82]:
reviews

[{'Product': 'samsung a8',
  'Name': 'Anoop Kumar Gupta',
  'Rating': 4,
  'Comment Heading': 'SOME IMPORTANT FLAWS, STILL A GREAT PHONE!!!',
  'Comments': "Firstly, I thank flipkart for its quick delivery in 5 days. Till date, I was using mobile phones below 20,000 Rs. This time however, I needed an allrounder phone having fast internet speed, dual sim slot, expandable memory, good camera and slim built and with one week of my experience I must say that Samsung Galaxy A8 has fulfilled most of the requirements. Usually smartphones having such a giant screen have average battery performance but samsung A8 battery is quite good. Well, I think I would mislead people if I don't mention the major flaws in this otherwise great phone :-\n    1) Heats up fast when charging & sometimes while using internet for a bit long time.\n    2) Use either second sim or memory card. No separate slot for memory card.\n    3) 2GB Ram gets easily occupied upto 75% because of preinstalled apps.\nI would recom

In [57]:
soup = BeautifulSoup(get_html("http://www.flipkart.com/apple-iphone-11-white-128-gb/product-reviews/itm846a8ba21ab11?pid=MOBFKCTS4MZVJ6FJ&aid=dc0f2b2d-7619-44cc-b5fc-2fe3e4d453a7&mid=FLIPKART&fid=033e8720-ce21-419d-aff9-21908cc1b2c9.MOBFKCTS4MZVJ6FJ&an=Camera&cat=Mobile&vert=Handset&page=15"))
mydict = soup.findAll(name="script",id="is_script")

In [59]:
json_text = re.search(r'^\s*window\.__INITIAL\_STATE__\s*=\s*({.*?})\s*;\s*$',
                              mydict[0].string, flags=re.DOTALL | re.MULTILINE).group(1)

script_dictionary = json.loads(json_text)

In [60]:
script_dictionary

{'abExperiments': {'isD2rCalloutApplicable': {'value': True,
   'abId': '41dee921'},
  'enableATCBasketBottomSheetPP': {'value': True, 'abId': '41dee921'},
  'basketOlpEnabled': {'value': True, 'abId': '41dee921'},
  'as-solr-bucket-pers': {'value': 'matching_noqv', 'abId': '7db3b04b'},
  'as-solr-bucket': {'value': 'matching_noqv', 'abId': '7db3b04b'},
  'as-solr-spell-bucket-pers': {'value': 'matching_noqv', 'abId': '7db3b04b'},
  'as-solr-spell-bucket': {'value': 'matching_noqv', 'abId': '7db3b04b'},
  'Autosuggest_cache': {'value': 'matching', 'abId': '7db3b04b'},
  'newap-homep-d9b50': {'value': 'newap-homep-d1ed4', 'abId': '8ca3ad43'},
  'newap-homep-55db3': {'value': 'newap-homep-78172', 'abId': '8ca3ad43'},
  'enableSavingsWidget': {'value': True, 'abId': '9cc1a74a'},
  'enableSavingsWidgetCart': {'value': True, 'abId': '9cc1a74a'},
  'babyc-produ-236fc': {'value': 'clone-produ-02c94', 'abId': '9cd5a9ab'},
  'toysa-produ-ad2d5': {'value': 'clone-produ-581ec', 'abId': '9cd5a9ab'

In [62]:
list_of_reviews = [value for value in script_dictionary['pageDataV4']['page']['data']['10002'] if value['slotType']=="WIDGET" if (value['widget']['type'] == "ASPECT_REVIEWS") or (value['widget']['type'] == "REVIEWS")]

for reviews in list_of_reviews:
    reviews['widget']['data']['renderableComponents'][0]['value']['author']
    reviews['widget']['data']['renderableComponents'][0]['value']['title']
    reviews['widget']['data']['renderableComponents'][0]['value']['rating']
    reviews['widget']['data']['renderableComponents'][0]['value']['text']

In [29]:
product_review

'https://www.flipkart.com/samsung-galaxy-a8-black-32-gb/product-reviews/itmedqx8qnafsqbc?pid=MOBE9YNTUZFBSYH5/samsung-galaxy-a8-black-32-gb/product-reviews/itmedqx8qnafsqbc?pid=MOBE9YNTUZFBSYH5&page=2'

In [28]:
script['pageDataV4']['page']['data']

{'ROOT': []}

In [1]:
"page{}".format(1)

'page1'

In [124]:
REVIEWS = {"title": "", "author":[]}

In [125]:
REVIEWS['title'] = "x"
REVIEWS['title'] = "y"

In [4]:
REVIEWS['author'].append("s")

In [126]:
REVIEWS

{'title': 'y', 'author': []}

In [6]:
REVIEWS['author'].append("t")

In [7]:
REVIEWS

{'title': '', 'author': ['s', 't']}